In [1]:
#import dependencies
import pandas as pd

In [2]:
#set path for csv file
csvpath = "Resources/purchase_data.csv"
#import the purchase_data.csv file as a DataFrame
purchase_data_df = pd.read_csv(csvpath)
purchase_data_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [3]:
#calculate the Player Count (PC)
total_players = len(purchase_data_df["SN"].unique())
total_players

576

In [4]:
#place Player Count into a summary DataFrame
summary_table_PC = pd.DataFrame({"Total Players": [total_players]})
summary_table_PC

,Total Players
0,576


In [5]:
#Purchasing Analysis (PA)
#need to find: number of unique items, average price, total # of purchases, total revenue
#be careful of ordering

#calculate the number of unique items
unique_items = len(purchase_data_df["Item ID"].unique())

#calculate the total revenue
total_revenue = sum(purchase_data_df["Price"])

#calculate the total number of purchases
total_number_of_purchases = len(purchase_data_df["Price"])

#calculate the average price 
average_price = total_revenue / total_number_of_purchases

In [6]:
# Place all of the Purchasing Analysis data found into a summary DataFrame
summary_table_PA = pd.DataFrame({"Number of Unique Items": [unique_items],
                                                  "Average Price": [average_price],
                                                  "Number of Purchases": [total_number_of_purchases],
                                                  "Total Revenue": [total_revenue]                                                  
                                                 })
summary_table_PA.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [7]:
#now format Average Price and Total Revenue in the summary table
summary_table_PA["Average Price"] = summary_table_PA["Average Price"].map("${:,.2f}".format)
summary_table_PA["Total Revenue"] = summary_table_PA["Total Revenue"].map("${:,.2f}".format)

summary_table_PA.head()

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


In [8]:
#Gender Demographics (total unique count of each gender and the gender's percent of total players)

#need to drop the duplicate players in order to find the numbers we're after
player_demo = purchase_data_df.loc[:, ["Gender", "SN", "Age"]]
player_demo = player_demo.drop_duplicates()

#generate totals for each gender
gender_total = player_demo["Gender"].value_counts()

#generate percents for each gender
gender_percent = gender_total / total_players 


In [9]:
#put gender demographics into a summary table
gender_demo_df = pd.DataFrame({"Total Count": gender_total, "Percentage of Players": gender_percent})
gender_demo_df

,Total Count,Percentage of Players
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [10]:
#format the percentage of players
gender_demo_df["Percentage of Players"] = gender_demo_df["Percentage of Players"].map("{:,.2%}".format)
gender_demo_df

,Total Count,Percentage of Players
Male,484,84.03%
Female,81,14.06%
Other / Non-Disclosed,11,1.91%


In [11]:
#Purchasing Analysis (Gender)

#use GroupBy in order to separate the data into fields according to "gender" values
gender_groupby_df = purchase_data_df.groupby(["Gender"])
gender_groupby_df.count()

,Purchase ID,SN,Age,Item ID,Item Name,Price
Gender,,,,,,
Female,113,113,113,113,113,113
Male,652,652,652,652,652,652
Other / Non-Disclosed,15,15,15,15,15,15


In [12]:
#by gender, obtain purchase count, avg. purchase price, total purchase value, avg. purchase total per person

#purchase count
purchase_count = gender_groupby_df["Purchase ID"].count()
purchase_count

total_purchased = gender_groupby_df["Price"].sum()
total_purchased

ave_price = total_purchased / purchase_count
ave_price

##need to find unique gender totals in order to find average total purchase per person
#total_purchased / comment above

Gender
Female                   3.203009
Male                     3.017853
Other / Non-Disclosed    3.346000
dtype: float64

In [13]:
#create new DataFrame to display Purchase Analysis by Gender
gender_groupby_df = pd.DataFrame({"Purchase Count": purchase_count,
                                  "Average Purchase Price": ave_price,
                                  "Total Purchase Value": total_purchased})
gender_groupby_df

,Purchase Count,Average Purchase Price,Total Purchase Value
Gender,,,
Female,113,3.203009,361.94
Male,652,3.017853,1967.64
Other / Non-Disclosed,15,3.346000,50.19


In [14]:
#Age Demographics

#figure out the minimum and maximum ages for players
print(player_demo["Age"].max())
print(player_demo["Age"].min())

45
7


In [15]:
#establish bins for ages
bins = [0, 9, 14, 19, 24, 29, 34, 39, 50]

group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

In [16]:
pd.cut(player_demo["Age"], bins, labels=group_labels).head()
#series

0    20-24
1      40+
2    20-24
3    20-24
4    20-24
Name: Age, dtype: category
Categories (8, object): [<10 < 10-14 < 15-19 < 20-24 < 25-29 < 30-34 < 35-39 < 40+]

In [17]:
player_demo["Age Group"] = pd.cut(player_demo["Age"], bins, labels=group_labels)
player_demo
#DF

,Gender,SN,Age,Age Group
0,Male,Lisim78,20,20-24
1,Male,Lisovynya38,40,40+
2,Male,Ithergue48,24,20-24
3,Male,Chamassasya86,24,20-24
4,Male,Iskosia90,23,20-24
...,...,...,...,...
773,Male,Hala31,21,20-24
774,Male,Jiskjask80,11,10-14
775,Female,Aethedru70,21,20-24
777,Male,Yathecal72,20,20-24


In [23]:
total_count_method1 = player_demo.loc[player_demo["Gender"].count()]
#DF

In [24]:
age_group = player_demo.loc[player_demo["Age Group"].value_counts()]
age_group
#DF

,Gender,SN,Age,Age Group
258,Male,Assehoan67,26,25-29
107,Male,Tyaerith73,36,35-39
77,Male,Lirtosia63,34,30-34
52,Male,Aelin32,20,20-24
31,Male,Farusrian86,37,35-39
22,Other / Non-Disclosed,Siarithria38,38,35-39
17,Male,Zontibe81,21,20-24
12,Male,Saelaephos52,21,20-24


In [25]:
#find out how many rows fall into each bin
print(age_group["Gender"].count())

8


In [28]:
total_count_method2 = age_group.count()
print(total_count_method2)
#series
#method 1 and 2, just dinking around, unsure what I was thinking

Gender       8
SN           8
Age          8
Age Group    8
dtype: int64


pandas.core.series.Series

In [29]:
total_players = player_demo["Age Group"].count()
total_players
#numpy.int32

numpy.int32

In [31]:
age_percent = total_count / total_players
age_percent
#DF

Gender       0.013889
SN           0.013889
Age          0.013889
Age Group    0.013889
dtype: float64

In [35]:
age_demo_df = pd.DataFrame({"Total Count": total_count_method1})
age_demo_df

,Total Count
Gender,Male
SN,Ilassa51
Age,20
Age Group,20-24


In [ ]:
total_players = player_demo["Age Group"].count()
total_players

In [ ]:
 # Create a GroupBy object based upon "View Group"
age_group = player_demo.groupby("Age Group")

# Find how many rows fall into each bin
#print(age_group["Gender"].count())

#total_count = age_group[["Age Group"]].count()
total_count = age_group.count()
total_count

In [ ]:
age_percent = total_count / total_players
age_percent

In [ ]:
age_demo_summary_table = pd.DataFrame({"Total Count": [total_count], "Percent": [age_percent]})
age_demo_summary_table

In [ ]:
#determine gender classifications
purchase_data_df["Gender"].unique()

In [ ]:
gender_groupby_df = purchase_data_df.groupby(["Gender"])
print(gender_groupby_df)
gender_groupby_df.count()

In [ ]:
gender_groupby_df["Purchase ID"].unique().count()


In [ ]:
gender_groupby_df["SN"].count()

In [ ]:
gender_groupby_df["SN"].unique()

In [ ]:
print(len(gender_groupby_df["SN"].unique()))

In [ ]:
male_players = purchase_data_df.loc[purchase_data_df["Gender"].value_counts()
total_male_players = len(male_players["SN"].unique())
print(total_male_players)

In [ ]:
gender_players_total = purchase_data_df["Gender"].value_counts()
gender_players_total

In [ ]:
#male_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Male", :]
#total_male_players = len(male_players["SN"].unique())
#total_male_players
#male_percent = (total_male_players) / (total_players) * 100
#male_percent

In [ ]:
#female_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Female", :]
#total_female_players = len(female_players["SN"].unique())
#total_female_players
#female_percent = (total_female_players) / (total_players) * 100
#female_percent

In [ ]:
#other_players = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed", :]
#total_other_players = len(other_players["SN"].unique())
#total_other_players
#other_percent = (total_other_players) / (total_players) * 100
#other_percent

In [ ]:
#gender_summary = pd.DataFrame([{"Total Count": total_male_players, "Percentage of Players": male_percent},
                               {"Total Count": total_female_players, "Percentage of Players": female_percent},
                               {"Total Count": total_other_players, "Percentage of Players": other_percent}])
#gender_summary

In [ ]:
gender_analysis_df = purchase_data_df[["SN", "Gender", "Item Name", "Price"]]
gender_analysis_df.head()

In [ ]:
gender_analysis_df = gender_analysis_df.groupby(["Gender"])

ga_test = gender_analysis_df.count()
ga_test

In [ ]:
gender_analysis_df["SN"].unique()

In [ ]:
#Gender Demographics (GD)
#find percent and count of: male players, female players and other/non-disclosed players

#male players
#create a dataframe removing females and other/non-disclosed
male_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Male", :]

#find unique number of male players
total_male_players = len(male_players_df["SN"].unique())

#percent of players which are male
male_percent = (total_male_players) / (total_players) * 100

male_players_df.head()



In [ ]:
#Gender Demographics (GD)
#female players

#create female dataframe
female_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Female", :]

#find total unique female players
total_female_players = len(female_players_df["SN"].unique())

female_percent = (total_female_players) / (total_players) * 100


female_players_df.head()

In [ ]:
#Gender Demographics (GD)
#other/non-disclosed players

#create other/non-disclosed dataframe
other_players_df = purchase_data_df.loc[purchase_data_df["Gender"] == "Other / Non-Disclosed", :]

#find total unique other players
total_other_players = len(other_players_df["SN"].unique())


other_percent = (total_other_players) / (total_players) * 100

other_players_df.head()

In [ ]:
m_f = pd.merge(male_players_df, female_players_df, on="Purchase ID")
gender_merge

In [ ]:
m_f_o = pd.merge(m_f, other_players_df, on="Purchase ID")
m_f_o